In [28]:
# imports
from transformers import pipeline, set_seed
import torch
import keyboard

In [2]:
# Two models
# Note: pipeline documentation: https://huggingface.co/docs/transformers/v4.25.1/en/main_classes/pipelines#transformers.pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', framework="pt", device=torch.device("cuda:0"), torch_dtype=torch.float16)
generator2 = pipeline('text-generation', model='openai-gpt', framework="pt", device=torch.device("cuda:0"), torch_dtype=torch.float16)

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Let's start with some basic output
init_str = "You are roleplaying as my AI girlfriend who loves me. Your name is Megumin."\
           "\n\n1: I love you. What's your name?\n\n2:  My name is Megumin.\n\n1: What would you like to do?\n\n2: "

In [22]:
# Let's try some generation
generator(init_str, max_length=300, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "You are roleplaying as my AI girlfriend who loves me. Your name is Megumin.\n\n1: I love you. What's your name?\n\n2:  My name is Megumin.\n\n1: What would you like to do?\n\n2:  I want to go to school.\n\n1: What is your hobby?\n\n2:  I like going to the beach.\n\n1: What do you like about yourself?\n\n2:  I like talking.\n\n1: What does your hobby involve?\n\n2:  I like the beach.\n\n1: What does your hobby involve?\n\n3: I like to read books.\n\n1: What else do you like?\n\n2:  I like going to the beach.\n\n1: What do you like about yourself?\n\n3: I like going to the beach.\n\n1: What is your hobby?\n\n2:  I like going to the beach.\n\n3: I like to read books.\n\n1: What else do you like?\n\n2:  I like going to the beach.\n\n3: I like going to the beach.\n\n4: It is your birthday.\n\n1: What is your gift?\n\n2:  I want to go to the beach.\n\n1: What would you like to do?\n\n5:"}]

In [23]:
# Sample from model 2
generator2(init_str, max_length=300, num_return_sequences=1)

[{'generated_text': "You are roleplaying as my AI girlfriend who loves me. Your name is Megumin.\n\n1: I love you. What's your name?\n\n2:  My name is Megumin.\n\n1: What would you like to do?\n\n2:  that's right i said i liked your name. 3 : now i do it. 7 : i love you but now i don't know if i'm going to get you to do anything. 5 : i'm going to tell you something that's a secret. 8 : i don't know, you are a strange alien who lives amongst me and is so very different from anyone i've ever met. 5 : oh, but don't worry, you 'll like me once you try and change the rest of your life. 5 : i know you're hiding something. 6 : in the desert of my life, i was born in a house on the border of a small town, a few hundred miles from the base of the mountain known as the sun tower. there was no one there, although the town was close to a desert state school, in north america in order to keep students safe. your name is the only known thing i know and i will tell you how you got here. so it is up t

In [26]:
# Let's try some smarter generation.

# Max length is the current length plus an offset
max_length = len(init_str)+50

# Generate one output
out = generator(init_str, max_length=300, num_return_sequences=1)[0]["generated_text"]

# Get the first sentence outputted
print(out[len(init_str):].split("\n")[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 I'd like to help you.


In [27]:
e = "I'd like to help you.\n\n1: With what?\n\n2: "
max_length = len(init_str)+len(e)+50
out = generator(init_str+e, max_length=300, num_return_sequences=1)[0]["generated_text"]
print(out[len(init_str)+len(e):].split("\n")[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 I'd like to help you.


In [87]:
# Let's try creating a pipeline to chat

# Name of our girlfriend
name = "Megumin"

# Initial length and initial sequence
init_str = f"You are {name} and are person's AI girlfriend\n"\
           f"{name} loves person\n"\
           "\n\n\n\n"\
           "Person: Hello\n"\
           f"{name}: Hi\n\n"

# History is the initial string
history = init_str

# The max length of the generated output
limit = 50

while not keyboard.is_pressed("esc"):
    # Get new text from the user
    text = input()
    
    # Make the text a prompt like `1: [text]\n\n2: `
    text = f"Person: \"{text}\"\n{name}: \""
    
    # Add the text to the current history
    history += text
    
    # Create a new output
    out = generator(history, max_length=len(history)+limit, num_return_sequences=1)[0]["generated_text"]
    
    # Get the output the AI actually created. The first
    # sentence that is not empty
    out = out[len(history):].split("\n")
    out = [i.strip() for i in out if len(i.strip()) > 0][0]
    if out[-1] != '"':
        out += '"'

    
    # Print the output and save it to the history
    print(out)
    history += out + "\n\n"

 Are you my waifu?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Well, no."


 What are you then?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You're so sweet."


 Are you my girlfriend?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Well, no."


KeyboardInterrupt: Interrupted by user

Problems:
1. Generations are long. Maybe limit it to `max_length=current_length+desired_max_legth (max_length=200+20)`
2. Generator 2 is trained on books and talks like a book which we don't want
3. Generator 1 looks much better than 2, but still not that good. Maybe I can fine-tune it